# **REGRESIÓN MULTIPLE**

In [42]:
import os
import pandas as pd
import pickle
import urllib.request
import kagglehub
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #Graficar
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import math
import logging
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white
from statsmodels.sandbox.stats.runs import runstest_1samp
import mlflow
from scipy.stats import kstest
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
import networkx as nx
import pickle
import pickle
from pathlib import Path
import xgboost as xgb



In [26]:
#Esta funcion va para ETL

def read_dataframe():
    path = kagglehub.dataset_download("ranjeetjain3/seaborn-tips-dataset")
    destino = "MLOPS_Project/data"
    shutil.copytree(path, destino, dirs_exist_ok=True)
    print("Dataset guardado en:", destino)
    datos = pd.read_csv("MLOPS_Project/data/tips.csv")
    return datos

datos = read_dataframe()
datos.head()
   

Dataset guardado en: MLOPS_Project/data


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [27]:
#Instalar el complementos
%pip install factor_analyzer
%pip install networkx
%pip install mlflow
%matplotlib inline

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: networkx in c:\users\eliana\appdata\local\programs\python\python313\lib\site-packages (3.5)




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
#Esta funcion va para Feature Ingineering

def normalizar_log(datos):
    continuas=datos[["total_bill","tip","size"]]
    continuas_L = pd.DataFrame()

    for col in continuas.columns:
        col_data = continuas[col]
        min_val = col_data.min()

        if (col_data < 0).any():
            desplazamiento = abs(min_val) + 1
            continuas_L[f"{col}_L"] = np.log(col_data + desplazamiento)

        elif (col_data == 0).any():
            continuas_L[f"{col}_L"] = np.log(col_data.replace(0, 1))
        else:
            continuas_L[f"{col}_L"] = np.log(col_data)

    return continuas_L

def concatenar(datos, continuas_L):
    datos_concat = pd.concat([datos, continuas_L], axis=1)
    return datos_concat

In [29]:

#Esta fraccion de codigo va para el main
continuas_L = normalizar_log(datos)
datos_final = concatenar(datos, continuas_L) # Este es el nuevo data set con toda la inforamcion completa
print(datos_final.head())

   total_bill   tip     sex smoker  day    time  size  total_bill_L     tip_L  \
0       16.99  1.01  Female     No  Sun  Dinner     2      2.832625  0.009950   
1       10.34  1.66    Male     No  Sun  Dinner     3      2.336020  0.506818   
2       21.01  3.50    Male     No  Sun  Dinner     3      3.044999  1.252763   
3       23.68  3.31    Male     No  Sun  Dinner     2      3.164631  1.196948   
4       24.59  3.61  Female     No  Sun  Dinner     4      3.202340  1.283708   

     size_L  
0  0.693147  
1  1.098612  
2  1.098612  
3  0.693147  
4  1.386294  


In [30]:
#PCA Será aplicado a variables con logaritmo
explicatorias=datos_final[["total_bill_L","tip_L","size_L"]]

def PCA_generation(explicatorias):

    scaler = StandardScaler()
    explicatorias_scaled = scaler.fit_transform(explicatorias)
    pca = PCA()
    res_pca = pca.fit(explicatorias_scaled)
    pred_pca = res_pca.transform(explicatorias_scaled)
    df_pred = pd.DataFrame(pred_pca, columns=[f"PC{i+1}" for i in range(pred_pca.shape[1])])
    componentes = df_pred[["PC1", "PC2"]].copy()

    cumVar = pd.DataFrame(np.cumsum(res_pca.explained_variance_ratio_)*100,
                        columns=["cumVarPerc"])
    expVar = pd.DataFrame(res_pca.explained_variance_ratio_*100, columns=["VarPerc"])
    pd.concat([expVar, cumVar], axis=1)\
        .rename(index={0: "PC1", 1: "PC2", 2: "PC3", 3: "PC4", 4: "PC5", 5: "PC6", 6: "PC7", 7: "PC8", 8: "PC9", 9: "PC10"})
    
    return componentes

In [31]:
def concatenar_PCA(datos_final, componentes):
    datos_concat = pd.concat([datos_final, componentes], axis=1)
    return datos_concat

In [32]:
componentes = PCA_generation(explicatorias)
datos_final = concatenar_PCA(datos_final, componentes)
print(datos_final.head())

   total_bill   tip     sex smoker  day    time  size  total_bill_L     tip_L  \
0       16.99  1.01  Female     No  Sun  Dinner     2      2.832625  0.009950   
1       10.34  1.66    Male     No  Sun  Dinner     3      2.336020  0.506818   
2       21.01  3.50    Male     No  Sun  Dinner     3      3.044999  1.252763   
3       23.68  3.31    Male     No  Sun  Dinner     2      3.164631  1.196948   
4       24.59  3.61  Female     No  Sun  Dinner     4      3.202340  1.283708   

     size_L       PC1       PC2  
0  0.693147 -1.712620  0.969288  
1  1.098612 -1.062917  1.360340  
2  1.098612  0.902003  0.106399  
3  0.693147  0.308759 -0.812399  
4  1.386294  1.647817  0.705635  


In [33]:
#Esta funcion va para ETL
def split_data(datos_final):
    np.random.seed(101)
    sample = np.random.choice(datos_final.index, size=int(0.8 * len(datos_final)), replace=False)
    train = datos_final.loc[sample]
    test = datos_final.drop(sample)
    #X_train = train[["total_bill","tip","size"]]
    X_train_L = train[["total_bill_L","tip_L","size_L", "PC1", "PC2"]]
    return X_train_L

In [34]:
datosEntrenamiento = split_data(datos_final) # Partiendo la data de entrenamiento (Dataframe)

In [35]:
datosEntrenamiento.head()

,total_bill_L,tip_L,size_L,PC1,PC2
151,2.574900,0.693147,0.693147,-1.172280,0.064256
34,2.878074,1.184790,0.693147,-0.105055,-0.718159
109,2.660959,1.386294,0.693147,-0.141578,-0.947225
4,3.202340,1.283708,1.386294,1.647817,0.705635
114,3.246880,1.386294,1.098612,1.357822,-0.138187


In [39]:
# Corrige la definición de la función para aceptar el argumento de entrada
def train_model_PCA(datosEntrenamiento):
    formula_PCA = 'tip_L ~ PC1 + PC2'
    model_PCA = sm.formula.ols(formula_PCA, data=datosEntrenamiento).fit()
    return model_PCA

In [37]:
modelo_entrenado_PCA = train_model_PCA(datosEntrenamiento) 
print(modelo_entrenado_PCA.summary())

                            OLS Regression Results                            
Dep. Variable:                  tip_L   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     920.8
Date:                Sun, 05 Oct 2025   Prob (F-statistic):           4.03e-99
Time:                        13:41:41   Log-Likelihood:                 116.09
No. Observations:                 195   AIC:                            -226.2
Df Residuals:                     192   BIC:                            -216.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9990      0.010    103.647      0.0

# **MLFLOW**

**Definir el servidor**

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
mlflow.search_experiments()

**Regresion Multiple con PCA**

In [40]:
experiment_name = "proyecto-propinas"
mlflow.set_experiment(experiment_name)
mlflow.autolog()

with mlflow.start_run(run_name="modelo_PCA") as run:
  
    formula_PCA = 'tip_L ~ PC1 + PC2'
    model_PCA = sm.formula.ols(formula_PCA, data=datosEntrenamiento).fit()

    # 📊 ANOVA para evaluar la significancia global del modelo
    aov_table = sm.stats.anova_lm(model_PCA, typ=2)
    display(aov_table)

    y_true = datosEntrenamiento['tip_L']
    y_pred = model_PCA.predict(datosEntrenamiento)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2_adj = model_PCA.rsquared_adj

    mlflow.log_metric("rmse", float(rmse))
    mlflow.log_metric("MSE", float(mse))
    mlflow.log_metric("MAE", float(mae))
    mlflow.log_metric("R2_adj", float(r2_adj))
    mlflow.log_metric("model_fvalue", float(model_PCA.fvalue))
    mlflow.log_metric("model_f_pvalue", float(model_PCA.f_pvalue))

    # 🧠 Interpretación automática del ANOVA
    print("\n🧠 Interpretación del ANOVA del modelo con PCA:")
    for index, row in aov_table.iterrows():
        if pd.notnull(row['PR(>F)']):
            p_value = row['PR(>F)']
            f_stat = row['F']
            print(f"\n🔹 Componente: {index}")
            print(f"   - Valor F: {f_stat:.2f}")
            print(f"   - Valor p: {p_value:.2e}")

            if p_value < 0.05:
                print("   ✅ Este componente es estadísticamente significativo para explicar 'tip_L'.")
                
            else:
                print("   ⚠️ Este componente NO es estadísticamente significativo.")
        else:
            print(f"\n🧾 Residual: Sin prueba de significancia (es la varianza no explicada del modelo).")

    
# Mostrar resumen del modelo
    summary_PCA = model_PCA.summary()
    print(summary_PCA)
  







2025/10/05 13:44:25 INFO mlflow.tracking.fluent: Experiment with name 'proyecto-propinas' does not exist. Creating a new experiment.
2025/10/05 13:44:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/10/05 13:44:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/10/05 13:44:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/10/05 13:44:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/05 13:44:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


,sum_sq,df,F,PR(>F)
PC1,26.321908,1.0,1455.997788,1.430241e-91
PC2,7.888629,1.0,436.359951,2.551693e-51
Residual,3.471026,192.0,NaN,NaN



🧠 Interpretación del ANOVA del modelo con PCA:

🔹 Componente: PC1
   - Valor F: 1456.00
   - Valor p: 1.43e-91
   ✅ Este componente es estadísticamente significativo para explicar 'tip_L'.

🔹 Componente: PC2
   - Valor F: 436.36
   - Valor p: 2.55e-51
   ✅ Este componente es estadísticamente significativo para explicar 'tip_L'.

🧾 Residual: Sin prueba de significancia (es la varianza no explicada del modelo).
                            OLS Regression Results                            
Dep. Variable:                  tip_L   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     920.8
Date:                Sun, 05 Oct 2025   Prob (F-statistic):           4.03e-99
Time:                        13:44:58   Log-Likelihood:                 116.09
No. Observations:                 195   AIC:                            -226.2
Df Residuals:                    

**Regresion Multiple sin PCA**

In [41]:
experiment_name = "proyecto-propinas"
mlflow.set_experiment(experiment_name)
mlflow.autolog()

# Finaliza cualquier run activa antes de iniciar una nueva
if mlflow.active_run() is not None:
    mlflow.end_run()

with mlflow.start_run(run_name="Reg_mult_sinPCA") as run:
    formula_m = 'tip_L ~ total_bill_L+size_L'
    model_m = sm.formula.ols(formula_m, data=datosEntrenamiento).fit()
    aov_table_m =  sm.stats.anova_lm(model_m,typ=2)
    display(aov_table_m)
    
    y_true = datosEntrenamiento['tip_L']
    y_pred = model_m.predict(datosEntrenamiento)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2_adj = model_m.rsquared_adj

    mlflow.log_metric("rmse", float(rmse))
    mlflow.log_metric("MSE", float(mse))
    mlflow.log_metric("MAE", float(mae))
    mlflow.log_metric("R2_adj", float(r2_adj))


# Mostrar resumen del modelo
    summary_m = model_m.summary()
    print(summary_m)

2025/10/05 15:49:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/10/05 15:49:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/10/05 15:49:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/10/05 15:49:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/10/05 15:49:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/10/05 15:49:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/05 15:49:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/05 15:49:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/05 15:49:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please

,sum_sq,df,F,PR(>F)
total_bill_L,7.157521,1.0,64.368522,9.983559e-14
size_L,0.445431,1.0,4.005820,4.675146e-02
Residual,21.349629,192.0,NaN,NaN


                            OLS Regression Results                            
Dep. Variable:                  tip_L   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     69.30
Date:                Sun, 05 Oct 2025   Prob (F-statistic):           2.20e-23
Time:                        15:49:38   Log-Likelihood:                -61.026
No. Observations:                 195   AIC:                             128.1
Df Residuals:                     192   BIC:                             137.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.7562      0.164     -4.602   